In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

C:\Users\84335\AppData\Local\Temp\ipykernel_10172\912164990.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\84335\AppData\Local\Temp\ipykernel_10172\912164990.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [3]:
spacex_df = pd.read_csv(path)
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
app = dash.Dash(__name__)
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

In [5]:
app.layout = html.Div(children=[html.H1('SpaceX launch Recoreds Dashboard',
                                        style={'textAlign': 'center',
                                               'color':'#503D36',
                                               'font-size':40}),
                                 # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                options = [
                                    {'label':'All Sites','value':'All Sites'},
                                    {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                    {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                    {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                    {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                ],
                                placeholder='Select a launch site here',
                                value='All sites',
                                searchable=True),
                                html.Br(),
                                #TASK 2: Add a pie chart to show the total successful launches count for all sites
                                html.Div(dcc.Graph(id='success_pie_chart')),
                                html.Br(),
                                
                                html.P('Payload range (Kg):'),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000,step=1000,
                                                value=[min_payload, max_payload],
                                                marks={0: '0', 2500:'2500',5000:'5000',
                                                       7500:'7500', 10000: '10000'}),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [6]:
### TASK 2
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id = 'success_pie_chart', component_property='figure'),
    Input(component_id = 'site-dropdown', component_property='value')
)
def get_pie_chart(launch_site):
    filtered_df = spacex_df
    if launch_site == 'All Sites':
        fig = px.pie(data_frame=filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches by Site')
        return fig
    else:
        specific_df = spacex_df.loc[spacex_df['Launch Site']==launch_site]
        piechart = px.pie(data_frame=specific_df, names='class',
                          title='Total Launch for a Specific Site')
        return piechart

In [7]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_payload_chart(launch_site, payload_slider):
    if launch_site =='All Sites':
        filtered_data = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0])
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        fig = px.scatter(data_frame=filtered_data, x='Payload Mass (kg)',
                         y='class', color='Booster Version Category')
        return fig
    else:
        filtered_data = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0])
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scatter_plot = px.scatter(data_frame=filtered_data, x='Payload Mass (kg)',
                             y='class', color='Booster Version Category')
        return scatter_plot

if __name__ == '__main__':
    app.run_server()